In [7]:
import pandas as pd
print("Welcome to my script! \nAuther: Zuoming Wang \nDate: 22th May, 2023")

Welcome to my script! 
Auther: Zuoming Wang 
Date: 22th May, 2023


In [8]:
# Prepare variables for loop

d0 = pd.DataFrame({"REPEAT":[],"VOLTAGE(V)":[],"CURRENT(A)":[]}).astype('float64')
size = 0
# print(d0)

In [9]:
area = 1.59*10**(-5)

In [10]:
# Loop
while True:
    src = input("\nPlease enter your input file name (Press 'enter' to finish): ")+'.csv' 
    
# Close the loop
    if src == '.csv':       
## Get size without duplicates in "REPEAT"
        n = len(d0[d0["REPEAT"] == 1]) # How many points in one single curve
        distinct = d0.drop_duplicates(subset=["REPEAT"])
        size = distinct.count().iloc[0] # How many curves are there
        break
        
# In the loop    
## Read file, select colums, edit"REPEAT"
    df = pd.read_csv(src, header=1).astype('float64')
    df = df[["REPEAT", "VOLTAGE(V)", "CURRENT(A)"]]
    
    print("Cut every curve to Voltage from -1V to 1V")
    
    df = df[df["VOLTAGE(V)"] >= -1.0019]
    df = df[df["VOLTAGE(V)"] <= 1]
    df = df.reset_index(drop=True)
    
    n = len(df[df["REPEAT"] == 1]) # How many points in one single curve
    size_df = len(df) // n # How many curves are in this .csv file
    

   
    if size_df * n != len(df):
        df = df[df["REPEAT"] != size_df + 1]
        print("Eliminate the incomplete repeat")
    # print(df.head(len(df)))
    print(f"{size_df} * {n}")
    
    df['REPEAT'] += size
                     
## Merge file
    d0 = pd.concat([d0,df], axis=0)
    
## Get size without duplicates in "REPEAT"
    distinct = d0.drop_duplicates(subset=["REPEAT"])
    size = distinct.count().iloc[0] # How many curves are there

Cut every curve to Voltage from -1V to 1V
20 * 101
Cut every curve to Voltage from -1V to 1V
20 * 101
Cut every curve to Voltage from -1V to 1V
Eliminate the incomplete repeat
492 * 101


In [11]:
# print(f"\nTotal:\n{d0.head(len(d0))}")

print(f'{size} * {n}')
print("\nMerge complete!")

532 * 101

Merge complete!


In [12]:
# Get the index of minimal "VOLTAGE(V)"  

edit_data = d0[d0['REPEAT'] == 1]["VOLTAGE(V)"]
x = abs(edit_data).idxmin(axis = 0, skipna = True) # x: The index of minimal "VOLTAGE(V)" 
print("Voltage = 0V, index = ",x)

Voltage = 0V, index =  50


In [13]:
# Get the index of "VOLTAGE(V)" = -1V & 1V

VR = d0[d0['VOLTAGE(V)'] == 1]["CURRENT(A)"].index.min()
print(f"Voltage = 1V,  index = {VR}")

VL = d0[d0['VOLTAGE(V)'] == -1]["CURRENT(A)"].index.min()
print(f"Voltage = -1V, index = {VL}")

Voltage = 1V,  index = 100
Voltage = -1V, index = 0


In [14]:
# Select retification factor

while True:
    num = 0 # How many curves need to be removed
    remove = [] # List for No.of curves that need to be removed
    RF_low = float(input("Please enter the minimal recitification factor: ")) # Retification factor
    RF_high = float(input("Please enter the maximal recitification factor: "))
    
    for row in range(0,size): # size: How many curves are there
        
            select_data = d0[d0['REPEAT'] == row+1]["CURRENT(A)"] # Select_data: Every data in a single curve (n = # points in a single curve)
            select_data = select_data.reset_index(drop=True)
#             select_data.index = [i for i in range(int(n))] # Reset the index of select_data
            select_data = select_data.rename(f'{str(row+1)}') # Rename each curve                       
            select_data += (0 - select_data[x]) # Edit every curve such that all curves pass through (0, 0) 
            
                                      
            if abs(select_data[VR])/abs(select_data[VL]) < RF_low or abs(select_data[VR])/abs(select_data[VL]) > RF_high:
                 a = int(select_data.name) 
                 remove.append(a) # Remove: Collect all curves that need to be removed in to a list               
                 num += 1 # Calculate how many curves are to be removed 
                    
#    print("\nEliminate data: ", remove)                
    print("Eliminated number: ", num)
    print("Rest total number: ", size - num)
    
    choice = input("Press 'enter' to continue\n(Or press 'space' + 'enter' if you want to re-define the recitification factor: ")
    if choice == " ":
        continue
    break

Eliminated number:  507
Rest total number:  25
Eliminated number:  507
Rest total number:  25
Eliminated number:  450
Rest total number:  82


In [15]:
# Please define your output file name

print("NOTICE: Please check is any .csv file open")
out_file = input('Please enter your output file name: ')

NOTICE: Please check is any .csv file open


In [16]:
# Remove data

for i in remove: # remove: List for No.of curves that need to be removed
    d0 = d0[d0.REPEAT != i] # Remove data with selected No.of curve
# print(f"After removing data: \n{d0.head(len(d0))}")
print(f"{size - num} * {n}")

82 * 101


In [17]:
# Create another dataframe: d1

d0.index = [i for i in range(len(d0))] # Reset the index of d0
d1 = d0[["VOLTAGE(V)"]].astype("float64")
d1 = d1[d1.index < int(n)]
# print(d1)

In [18]:
d2 = d1

In [19]:
# Get a valid "REPEAT" value

valid = (d0["REPEAT"]).iloc[0]
print(f"The first valid 'REPEAT' = {valid}" )

The first valid 'REPEAT' = 1.0


In [20]:
# chage unit, divided by area
# d0['Cal'] = abs(d0["VOLTAGE(V)"] ** 2 / d0["CURRENT(A)"]) * area # V^2/(A/cm2)

d0["CURRENT(A)"] = d0["CURRENT(A)"] * 10**6 # [A] -> [uA]
d0.columns = ['REPEAT','VOLTAGE(V)','CURRENT(uA)']
print("unit = [uA]")
# print(d0.head())

unit = [uA]


In [21]:
# Get the index of minimal abs("VOLTAGE(V)") 

edit_data = d0[d0['REPEAT'] == valid]["VOLTAGE(V)"]
# print(d0.head())
x = abs(edit_data).idxmin(axis = 0, skipna = True) # x: The index of minimal "VOLTAGE(V)"

In [22]:
# Split and transfer data to d1 & delete data in d0

print('Organizing data...\nPlease be patient, this process may take a few minutes')


index = 0

for i in range(size - num):

    select_group = d0.drop_duplicates(subset=["VOLTAGE(V)"]).astype('float64') # type(select_group) = dataframe
    # print(select_group.head())
    
    repeat = int(select_group.at[index,"REPEAT"]) # Extract every same "REPEAT" in one run

        
    select_data = select_group["CURRENT(uA)"] # Choose data with the same "REPEAR"     
    select_data = select_data.rename(f'{str(repeat)}') # Rename columns
    select_data = select_data.reset_index(drop=True)
    select_data += (0 - select_data[x]) # Edit every curve such that all curves pass through (0, 0) 
    d1 = pd.concat([d1, select_data], axis = 1) # Transfer data to d1
    

    select_data2 = abs(edit_data ** 2 / select_data) * area * 10 ** 6
    select_data2 = select_data2.rename(f'{str(repeat)}') # Rename columns
#     select_data2 = select_data2.reset_index(drop=True)
    d2 = pd.concat([d2, select_data2], axis = 1) # Transfer data to d1   
    
    
       
    d0 = d0[d0.REPEAT != repeat] # Eliminate data from d0
    
    index += int(n) # Adding index to points that belong to next curve
    

    


Organizing data...
Please be patient, this process may take a few minutes


In [23]:
# d2.fillna(0,inplace=True)
    
    
    
# print(d1.head(len(d1)))
# print(d2.head(len(d1)))

In [24]:
##### Export dataframe to .csv

d1.to_csv(out_file + "_forOrigin.csv", index=False, header=True)
d2.to_csv(out_file + "_forOrigin2.csv", index=False, header=True)
print("Complete! \n[Voltage]=V, [Current]=uA")

Complete! 
[Voltage]=V, [Current]=uA


In [25]:
# Choose if restart
import os
import sys
choice = input("Restart? (y/n)? ")
def restart_program():
    python = sys.executable
    os.execl(python, python, * sys.argv)
if choice == 'y':
    print('restart...')
    restart_program()